# Probar Acceso a Datos

In [1]:
import os
import pandas as pd
import plotly.express as px
import streamlit as st
import sys
import os

# Añadir el path raíz del proyecto
project_root = os.path.abspath(os.path.join(os.getcwd()))
sys.path.append(project_root)
# Subir un nivel desde el directorio actual del notebook
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)

# Verifica que 'modules' esté en el path
print(sys.path)

from modules.db import get_sqlserver_engine, get_pg_engine
from modules.ui import render_header, make_date_filters
from modules.queries import (
    SQL_SGM_KIKKER_VS_OC_MENSUAL,
    SQL_SGM_KIKKER_DETALLE,
    SQL_SGM_KIKKER_DUP,
    SQL_PG_KIKKER_MENSUAL,
    SQL_C_COMPRA_KIKKERP_GEN,
    SQL_C_COMPRA_KIKKER_GEN,
)

# Imprimir directorio actual para depuración
print(f"Directorio actual: {os.getcwd()}") 

# from modules.queries import (
#     SQL_OC_GENERADAS_RANGO,
#     SQL_OC_GENERADAS_RANGO_EXT,
#     SQL_RANKING_COMPRADORES,
#     SQL_RANKING_COMPRADORES_NOMBRE,
# )

# Imprimir directorio actual para depuración
print(f"Directorio actual: {os.getcwd()}")  


['C:\\Users\\eduar\\AppData\\Local\\Programs\\Python\\Python311\\python311.zip', 'C:\\Users\\eduar\\AppData\\Local\\Programs\\Python\\Python311\\DLLs', 'C:\\Users\\eduar\\AppData\\Local\\Programs\\Python\\Python311\\Lib', 'C:\\Users\\eduar\\AppData\\Local\\Programs\\Python\\Python311', 'e:\\ETL\\connexa_monitor\\.venv', '', 'e:\\ETL\\connexa_monitor\\.venv\\Lib\\site-packages', 'e:\\ETL\\connexa_monitor\\.venv\\Lib\\site-packages\\win32', 'e:\\ETL\\connexa_monitor\\.venv\\Lib\\site-packages\\win32\\lib', 'e:\\ETL\\connexa_monitor\\.venv\\Lib\\site-packages\\Pythonwin', 'e:\\ETL\\connexa_monitor\\pages', 'e:\\ETL\\connexa_monitor']
186.158.182.54
10.54.200.92
Directorio actual: e:\ETL\connexa_monitor\pages
Directorio actual: e:\ETL\connexa_monitor\pages


In [ ]:

import os
from functools import lru_cache
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
from datetime import timedelta

load_dotenv()
print (os.getenv("PG_HOST","no-pg-host"))
print (os.getenv("SQL_SERVER","no-SQL-host"))

host = os.getenv("PG_HOST","186.158.182.54")
port = os.getenv("PG_PORT","5432")
db   = os.getenv("PG_DB","diarco_data")
user = os.getenv("PG_USER","postgres")
pw   = os.getenv("PG_PASSWORD","postgres")

print(f"Conectando a PG en {host}:{port} a la base {db} con usuario {user}")

186.158.182.54
10.54.200.92
Conectando a PG en 186.158.182.54:5432 a la base diarco_data con usuario postgres


In [3]:
def fetch_sgm_mensual(desde, hasta):
    eng = get_sqlserver_engine()
    if eng is None: return pd.DataFrame()
    with eng.connect() as con:
        df = pd.read_sql(SQL_SGM_KIKKER_VS_OC_MENSUAL, con, params={"desde": desde, "hasta": hasta})
    return df

def fetch_sgm_detalle(desde, hasta):
    eng = get_sqlserver_engine()
    if eng is None: return pd.DataFrame()
    with eng.connect() as con:
        df = pd.read_sql(SQL_SGM_KIKKER_DETALLE, con, params={"desde": desde, "hasta": hasta})
    return df

def fetch_pg_kikker_mensual(desde, hasta):
    eng = get_pg_engine()
    with eng.connect() as con:
        df = pd.read_sql(SQL_PG_KIKKER_MENSUAL, con, params={"desde": desde, "hasta": hasta})
    return df

def fetch_sgm_dup(desde, hasta):
    eng = get_sqlserver_engine()
    if eng is None: return pd.DataFrame()
    with eng.connect() as con:
        df = pd.read_sql(SQL_SGM_KIKKER_DUP, con, params={"desde": desde, "hasta": hasta})
    return df

In [4]:
desde = pd.to_datetime("2025-07-01").date()
hasta = pd.to_datetime("2025-08-31").date()


In [5]:
sgm_d = fetch_sgm_detalle(desde, hasta)
pg_m  = fetch_pg_kikker_mensual(desde, hasta)
dup   = fetch_sgm_dup(desde, hasta)

In [8]:
if sgm_d.empty:
        st.info("Sin detalle SGM para reconciliar.")
else:
    # set de KIKKER en SGM
    eng_pg = get_pg_engine()
    hasta_mas_1 = hasta + timedelta(days=1) # type: ignore

    with eng_pg.connect() as con:
        df_pg_k = pd.read_sql(SQL_C_COMPRA_KIKKER_GEN, con, params={"desde": desde, "hasta_mas_1": hasta_mas_1}) # type: ignore


    sgm_k = sgm_d[["C_COMPRA_KIKKER"]].dropna().drop_duplicates().rename(columns={"C_COMPRA_KIKKER":"kikker"})
    recon = sgm_k.merge(df_pg_k, on="kikker", how="outer", indicator=True)
    recon["estado"] = recon["_merge"].map({
        "both": "OK (1:1)",
        "left_only": "SGM sin CONNEXA",
        "right_only": "CONNEXA sin SGM",
    })
    kpis = recon["estado"].value_counts().to_frame("kikkers").reset_index().rename(columns={"index":"estado"})


In [ ]:
# set de KIKKER en SGM


eng_pg = get_pg_engine()
hasta_mas_1 = hasta + timedelta(days=1)

with eng_pg.connect() as con:
    df_pg_k = pd.read_sql(SQL_C_COMPRA_KIKKER_GEN, con, params={"desde": desde, "hasta_mas_1": hasta_mas_1})

print(f"Encontrados {len(df_pg_k)} KIKKER únicos en PG entre {desde} y {hasta}")



Encontrados 37 KIKKER únicos en PG entre 2025-07-01 y 2025-08-31
